<a href="https://colab.research.google.com/github/Vanessa4422/Digital-dashbaord/blob/main/ScratchDash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q jupyter-dash==0.3.0rc1 dash-bootstrap-components transformers

     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 194kB 10.4MB/s 
     |████████████████████████████████| 1.4MB 9.6MB/s 
     |████████████████████████████████| 81kB 10.2MB/s 
     |████████████████████████████████| 2.9MB 41.5MB/s 
     |████████████████████████████████| 890kB 45.3MB/s 
     |████████████████████████████████| 1.0MB 45.5MB/s 
     |████████████████████████████████| 3.5MB 41.2MB/s 
     |████████████████████████████████| 194kB 47.6MB/s 
     |████████████████████████████████| 1.8MB 43.3MB/s 
     |████████████████████████████████| 358kB 41.4MB/s 


In [4]:
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import folium

app = JupyterDash(__name__, suppress_callback_exceptions=True)

# STYLING
Index_STYLE = {
    "position": "fixed",
    "top": 8.2,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#2b2b2b",

}

CONTENT_STYLE = {
    "background": '#1f1f1f',
    "top": 0,
    "margin-left": "17rem",
    "margin-right": "8rem",
    "padding": "2rem 1rem",
    "textAlign": "center",
    "color":"#999"
}

sidebar = html.Div([
    
    html.H1('FCDO Covid-19 Dashboard'),
    html.Br(),
    html.Hr(),
    dbc.Nav([
    dbc.NavLink('Internal Reporting', href='/page-1', style={'color':'#999'}),
    html.Br(), html.Br(),
    dbc.NavLink('Reporting in Open source', href='/page-2', style={'color':'#999'}),
    html.Br(), html.Br(),
    dbc.NavLink('Media Reporting', href='/page-3',style={'color':'#999'}),
    html.Br(), html.Br(),
    dbc.NavLink('UK Covid-19 summary', href='/page-4', style={'color':'#999'})
    ])
], style = Index_STYLE)

home_page = html.Div([
    
    html.H1('FCDO Covid-19 Dashboard'),
    html.Br(),
    html.Br(),
    html.Hr(),
], style = CONTENT_STYLE)
    

# Page 1 Layout sitrep map

my_map = folium.Map(location=[51.5074, 0.1278])
page_1_layout = html.Div([
    html.H1('Internal Reporting'),
    html.P('Visualising FCDO Covid-19 Sitreps using an interactive world map'),
    html.Hr(),
    html.Iframe(id='my_map'),   
      
    ], style = CONTENT_STYLE)

@app.callback(Output('page-1-content', 'children'),
              [Input('page-1-dropdown', 'value')])
def page_1_dropdown(value):
    return 'You have selected "{}"'.format(value)

# Page 2 layout open source

# Data 

def prepare_daily_report():

    current_date = (datetime.today() - timedelta(days=1)).strftime('%m-%d-%Y')

    df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/' + current_date + '.csv')
    df_country = df.groupby(['Country_Region']).sum().reset_index()
    df_country.replace('US', 'United States', inplace=True)
    df_country.replace(0, 1, inplace=True)
    
    code_df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
    df_country_code = df_country.merge(code_df, left_on='Country_Region', right_on='COUNTRY', how='left')

    df_country_code.loc[df_country_code.Country_Region == 'Congo (Kinshasa)', 'CODE'] = 'COD'
    df_country_code.loc[df_country_code.Country_Region == 'Congo (Brazzaville)', 'CODE'] = 'COG'
    
    return(df_country_code)

page_2_layout = html.Div([
    html.H1('Reporting in open source'),
    html.H3("Covid-19 Impact by country"),
    html.Hr(),
    html.Div([html.Div([html.Span("Metric to display : ", className="six columns",
                                           style={"text-align": "center", "width": "40%", "padding-top": 10}),
                                 dcc.Dropdown(id="value-selected", value='Confirmed',
                                              options=[{'label': "Confirmed ", 'value': 'Confirmed'},
                                                       {'label': "Recovered ", 'value': 'Recovered'},
                                                       {'label': "Deaths ", 'value': 'Deaths'},
                                                       {'label': "Active ", 'value': 'Active'}],
                                              style={"display": "block", "margin-left": "auto", "margin-right": "auto",
                                                     "width": "70%"},
                                              className="six columns")], className="row"),
                       dcc.Graph(id="my-graph")
                       ], className="container"),
     html.H2("Our World in Data Covid-19"),
     html.Iframe(src="https://ourworldindata.org/coronavirus-data-explorer?zoomToSelection=true&time=2020-03-01..latest&country=IND~USA~GBR~CAN~DEU~JPN&region=World&casesMetric=true&interval=smoothed&perCapita=true&smoothing=7&pickerMetric=new_cases_per_million&pickerSort=desc", style={"width":"80%", "height":"600px"}),
    ], style=CONTENT_STYLE)


# Map callbacks 
@app.callback(
    Output("my-graph", "figure"),
    Input("value-selected", "value"))

def update_figure(selected):
#dff = prepare_confirmed_data()

    dff = prepare_daily_report()
    dff['hover_text'] = dff["Country_Region"] + ": " + dff[selected].apply(str)

    trace = go.Choropleth(locations=dff['CODE'],z=np.log(dff[selected]),
                          text=dff['hover_text'],
                          hoverinfo="text",
                          marker_line_color='white',
                          autocolorscale=False,
                          reversescale=True,
                          colorscale="greys",marker={'line': {'color': 'rgb(180,180,180)','width': 0.5}},
                          colorbar={"thickness": 10,"len": 0.3,"x": 0.9,"y": 0.7,
                                    'title': {"text": 'persons', "side": "bottom"},
                                    'tickvals': [ 2, 10],
                                    'ticktext': ['100', '100,000']})   
  

    return {"data": [trace],
            "layout": go.Layout(height=600, width= 900, geo_bgcolor="#1f1f1f", paper_bgcolor= 'rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', margin=dict(l=1, r=6, b=50,t= 50,pad=2),                                            
                                                                                      
                                geo={'showframe': False,
                                    'showcoastlines': False,
                                    'projection': {'type': "miller"}
                                                })}

                      



@app.callback(Output('page-2-content', 'children'),
              [Input('page-2-radios', 'value')])
def page_2_radios(value):
    return 'You have selected "{}"'.format(value)

# Page 3 layout Media Reporting 
page_3_layout = html.Div([
    html.H1('Media Reporting'),
    html.P('Media updates on Covid-19, including trending topics'),
    html.Hr(),
    html.Div(id='page-3-content')
],style = CONTENT_STYLE)

@app.callback(Output('page-3-content', 'children'),
              [Input('page-3-radios', 'value')])
def page_2_radios(value):
    return 'You have selected "{}"'.format(value)

# Page 4 layout UK covid Summary 
page_4_layout = html.Div([
    html.H1('UK Covid-19 Summary'),
    html.P('Data and insights on the UK domestic picture'),
    html.Hr(),
    html.Iframe(src="https://www.ons.gov.uk/visualisations/dvc914/map/index.html",
                style={"height": "600px", "width": "80%"}),
    html.Div(id='page-4-content'),
    ],style = CONTENT_STYLE)

@app.callback(Output('page-4-content', 'children'),
              [Input('page-4-radios', 'value')])
def page_2_radios(value):
    return 'You have selected "{}"'.format(value)

# Update the index
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/page-1':
        return page_1_layout
    elif pathname == '/page-2':
        return page_2_layout
    elif pathname == '/page-3':
        return page_3_layout
    elif pathname == '/page-4':
        return page_4_layout
    else:
        return home_page
    
app.layout = html.Div([
    dcc.Location(id='url', refresh=False), sidebar,
    html.Div(id='page-content')
])

if __name__ == '__main__':
    app.run_server(mode='inline')

<IPython.core.display.Javascript object>